In [1]:
import numpy as np
import tensorflow as tf

In [2]:
!pip install -q tensorflow-hub
!pip install -q tfds-nightly

In [3]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [4]:
print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전:  2.3.1
즉시 실행 모드:  True
허브 버전:  0.9.0
GPU 사용 불가능


In [5]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [13]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(1)))
type(train_examples_batch)
print(train_examples_batch)

tf.Tensor([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."], shape=(1,), dtype=string)


In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [8]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                            dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [9]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [12]:
history = model.fit(train_data.shuffle(10000).batch(512),
                  epochs=20,
                  validation_data=validation_data.batch(512),
                  verbose=1) #verbose 0:silent, 1:progress bar, 2:one line per epoch

Epoch 1/20
30/30 [==============================] - 2s 64ms/step - loss: 0.8626 - accuracy: 0.5591 - val_loss: 0.6545 - val_accuracy: 0.6316
Epoch 2/20
30/30 [==============================] - 2s 56ms/step - loss: 0.6208 - accuracy: 0.6338 - val_loss: 0.6014 - val_accuracy: 0.6387
Epoch 3/20
30/30 [==============================] - 2s 55ms/step - loss: 0.5806 - accuracy: 0.6689 - val_loss: 0.5746 - val_accuracy: 0.6638
Epoch 4/20
30/30 [==============================] - 2s 56ms/step - loss: 0.5529 - accuracy: 0.6875 - val_loss: 0.5526 - val_accuracy: 0.7026
Epoch 5/20
30/30 [==============================] - 2s 58ms/step - loss: 0.5275 - accuracy: 0.7161 - val_loss: 0.5299 - val_accuracy: 0.7151
Epoch 6/20
29/30 [============================>.] - ETA: 0s - loss: 0.5019 - accuracy: 0.7368

KeyboardInterrupt: 

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))